# Deploying an MLflow trained model using Sagemkaer

## Prep env

### Install mlflow and SageMaker packages

In [ ]:
!pip install mlflow
!pip install awscli
!pip install boto3

In [1]:
!aws sts get-caller-identity

{
    "UserId": "AIDAWBMCXFXXOLO77UXD4",
    "Account": "415275363822",
    "Arn": "arn:aws:iam::415275363822:user/qa"
}


In [2]:
import boto3
client = boto3.client("sts")
client.get_caller_identity()["Arn"]

'arn:aws:iam::415275363822:user/qa'

### Creat a SageMaker
Navigate to AWS console and choose Identity and Access Management tool (IAM)”, where it provides the required permissions to deploy and use models to users.

* Create a user with specific set of permissions for model deployment.
    * Add another 3 policies to this user:
        1. AmazonEC2ContainerRegistryFullAccess; 
        2. AmazonSageMakerFullAccess; 
        3. S3_Full_Access_Policy (this is manually created)
* Create a S3 bucket save our model artifacts
* Create Access Key from AWS Console or CLI
* Create a role to acces AWS service such as SageMaker
* Install Docker engine to containerize the model

In [3]:
!aws iam list-roles| grep -i sagemaker| grep Arn

            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSageMaker-ExecutionRole-20220118T174575",
            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSagemakerCanvasForecastRole-20230728T085948",
            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSagemakerCanvasForecastRole-DataScientist1",
            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSageMakerServiceCatalogProductsApiGatewayRole",
            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSageMakerServiceCatalogProductsCloudformationRole",
            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSageMakerServiceCatalogProductsCodeBuildRole",
            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSageMakerServiceCatalogProductsCodePipelineRole",
            "Arn": "arn:aws:iam::415275363822:role/service-role/AmazonSageMakerServiceCatalogProductsEventsRole",
            "Arn": "arn:aws:iam::415275363822:role/service-role/Am

## Train a sample model using Mlflow

In [4]:
import mlflow
import mlflow.sklearn

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=50000, n_features=3, n_informative=3, n_redundant=0, n_classes=2, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

mlflow.set_experiment('classification_model')

with mlflow.start_run(run_name='My model experiment') as run:
    
    # add parameters for tuning
    c = 0.1
    solver = 'liblinear'
    
    # Log parameters to MLFlow
    mlflow.log_param('c', c)
    mlflow.log_param('solver', solver)

    # train the model
    lr = LogisticRegression(C = c, solver = solver)
    lr.fit(X_train, y_train)
    predictions = lr.predict(X_test)

    # Log the model to MLFlow
    mlflow.sklearn.log_model(lr, 'logistic-regression-model')
    
    # log model performance to MLFlow
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric('mse', mse)
    print('mse: %f' % mse)

mse: 0.106600


### Export the training datatset to a csv for future experiments (Optional)

In [5]:
import numpy as np
import pandas as pd

# Create a DataFrame with X_train and y_train
data = np.hstack((X_train, y_train.reshape(-1, 1)))
df = pd.DataFrame(data)

# Define the file path
train_data_file_path = 'mlflow_train_data.csv'

# Save the DataFrame as a CSV file
df.to_csv(train_data_file_path, index=False, header=False)

### Access mlflow via UI

In [57]:
!mlflow ui --port 7868 --host 0.0.0.0

/home/alfred/anaconda3/envs/dui/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/alfred/anaconda3/envs/dui/lib/python3.11/site-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2023-10-26 18:47:31 +0000] [11988] [INFO] Starting gunicorn 21.2.0
[2023-10-26 18:47:31 +0000] [11988] [INFO] Listening at: http://0.0.0.0:7868 (11988)
[2023-10-26 18:47:31 +0000] [11988] [INFO] Using worker: sync
[2023-10-26 18:47:31 +0000] [11989] [INFO] Booting worker with pid: 11989
[2023-10-26 18:47:31 +0000] [11990] [INFO] Booting worker with pid: 11990
[2023-10-26 18:47:32 +0000] [11991] [INFO] Booting worker with pid: 11991
[2023-10-26 18:47:32 +000

In [58]:
!cd /home/alfred/databricks-ml-examples/mlruns/546196553103136246/8bc0f39b38404cb1b68d0c64d71df4cd/artifacts/logistic-regression-model

## Push the trained model to ECR 

In [ ]:
!mlflow sagemaker build-and-push-container

......

Successfully built b06895e6648a
Successfully tagged mlflow-pyfunc:latest
2023/10/23 19:03:35 INFO mlflow.sagemaker: Pushing image to ECR
2023/10/23 19:03:35 INFO mlflow.sagemaker: Pushing docker image mlflow-pyfunc to 415275363822.dkr.ecr.us-west-2.amazonaws.com/mlflow-pyfunc:2.7.1
2023/10/23 19:03:36 INFO mlflow.sagemaker: Created new ECR repository: mlflow-pyfunc
2023/10/23 19:03:36 INFO mlflow.sagemaker: Executing: aws ecr get-login-password | docker login  --username AWS --password-stdin 415275363822.dkr.ecr.us-west-2.amazonaws.com;
docker tag mlflow-pyfunc 415275363822.dkr.ecr.us-west-2.amazonaws.com/mlflow-pyfunc:2.7.1;
docker push 415275363822.dkr.ecr.us-west-2.amazonaws.com/mlflow-pyfunc:2.7.1
WARNING! Your password will be stored unencrypted in /home/alfred/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
The push refers to repository [415275363822.dkr.ecr.us-west-2.amazonaws.com/mlflow-pyfunc]
dabb74b7228c: Pushed 
fb207a1e2297: Pushed 
c7d2c1b01e89: Pushed 
d83ee2f0e942: Pushed 
cb73c5f95ea3: Pushed 
b058a144849e: Pushed 
ab3377661fad: Pushed 
86a2582c70b8: Pushed 
8d8fc1d7c90e: Pushed 
d083867fc724: Pushed 
b17b4398c6d2: Pushed 
a46e51725894: Pushed 
9c11ebf3daf1: Pushed 
54da92f070b6: Pushed 
0d72c8d03be4: Pushed 
f24e5b689ad1: Pushed 
dc0fc6335310: Pushed 
074cb65e9492: Pushed 
e1f8c00343c5: Pushed 
6c3e7df31590: Pushed 
2.7.1: digest: sha256:c8e7266df9c8646c6f3135f98f4ae9d090a4de79c4c1c328ca448bea3bf77266 size: 4529

In [32]:
!docker images

REPOSITORY                                                              TAG                            IMAGE ID       CREATED         SIZE
415275363822.dkr.ecr.us-west-2.amazonaws.com/mlflow-pyfunc              2.7.1                          b06895e6648a   45 hours ago    2.62GB
mlflow-pyfunc                                                           latest                         b06895e6648a   45 hours ago    2.62GB
ubuntu                                                                  20.04                          bf40b7bc7a11   3 weeks ago     72.8MB
gsa                                                                     v0                             7b171a5c3709   7 weeks ago     19.2GB
ghcr.io/huggingface/text-generation-inference                           1.0.2                          ff5ecf561e97   2 months ago    10.1GB
415275363822.dkr.ecr.us-west-2.amazonaws.com/sagemaker-decision-trees   latest                         0e43a9b29b04   2 months ago    380MB
sagemaker-decisi

In [33]:
!aws ecr describe-repositories

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-west-2:415275363822:repository/sagemaker-decision-trees",
            "registryId": "415275363822",
            "repositoryName": "sagemaker-decision-trees",
            "repositoryUri": "415275363822.dkr.ecr.us-west-2.amazonaws.com/sagemaker-decision-trees",
            "createdAt": "2023-07-28T18:45:47+00:00",
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            },
            "encryptionConfiguration": {
                "encryptionType": "AES256"
            }
        },
        {
            "repositoryArn": "arn:aws:ecr:us-west-2:415275363822:repository/sagemaker-snowflake-workshop",
            "registryId": "415275363822",
            "repositoryName": "sagemaker-snowflake-workshop",
            "repositoryUri": "415275363822.dkr.ecr.us-west-2.amazonaws.com/sagemaker-snowflake-workshop",
            "createdAt": "2023-

In [ ]:
!aws ecr describe-images --repository-name mlflow-pyfunc

{
    "imageDetails": [
        {
            "registryId": "415275363822",
            "repositoryName": "mlflow-pyfunc",
            "imageDigest": "sha256:c8e7266df9c8646c6f3135f98f4ae9d090a4de79c4c1c328ca448bea3bf77266",
            "imageTags": [
                "2.7.1"
            ],
            "imageSizeInBytes": 1090875471,
            "imagePushedAt": "2023-10-23T19:04:26+00:00",
            "imageManifestMediaType": "application/vnd.docker.distribution.manifest.v2+json",
            "artifactMediaType": "application/vnd.docker.container.image.v1+json"
        }
    ]
}

## Push a mlflow model to a SageMaker Model Registry
Push an MLflow model to Sagemaker model registry. Current active AWS account needs to have correct permissions setup.

## Deploy model using SageMaker

As we have the container ready, we can deploy a model to the Sagemaker.

To keep all the deployment details in one place I‘m’ creating a new file “deployment.py” in the folder with the model training file.

It’s almost fully copy-pasted from the MLFlow documentation, and you can refer to it.

Now all what we have to do is provide mlflow our image URL and desired model and then we can deploy these models to SageMaker.
Important: Before doing all following steps, you should create a special CLI token between your terminal and AWS. For this boto3 package is responsible, and you can do it in very simple way, just by setting your environment variables in your terminal like following:

Type AWS_ACCESS_KEY=XXXXXXXXX, where XXXXXXXXX is your AWS Access Key for your User in AWS.
Type AWS_SECRET_ACCESS_KEY=XXXXXXXXX, where XXXXXXXXX is your AWS Secret Access Key for your User in AWS.
Ocje you set your virtual environment, the terminal with active CLI is able to communicate with AWS services and make required operations.
Create a new Python script in your root project directory by terminal command touch deploy.py. This command will create a new file deploy.py.
Write the following script logic in the deploy.py you have just created.

<b>model_uri</b> – tag supports the following formats

* The location, in URI format, of the MLflow model to deploy to SageMaker. For example:
* /Users/me/path/to/local/model
* relative/path/to/local/model
* s3://my_bucket/path/to/model
* runs:/<mlflow_run_id>/run-relative/path/to/model
* models:/<model_name>/<model_version>
* models:/<model_name>/<stage>

In [28]:
from mlflow.deployments import get_deploy_client

experiment_id = '546196553103136246'
run_id = '8bc0f39b38404cb1b68d0c64d71df4cd'
model_name = 'logistic-regression-model'
region = 'us-west-2'
aws_id = '415275363822'
user_arn = f'arn:aws:iam::{aws_id}:user/qa'
exec_arn = f'arn:aws:iam::{aws_id}:role/service-role/AmazonSageMaker-ExecutionRole-20220118T174575'
deployment_name = 'mlflow-sagemaker-deploy-02'
model_uri = '/home/alfred/databricks-ml-examples/mlruns/%s/%s/artifacts/%s' % (experiment_id, run_id, model_name)
tag_id = '2.7.1'

image_url = aws_id + '.dkr.ecr.' + region + '.amazonaws.com/mlflow-pyfunc:' + tag_id

In [29]:
model_uri

'/home/alfred/databricks-ml-examples/mlruns/546196553103136246/8bc0f39b38404cb1b68d0c64d71df4cd/artifacts/logistic-regression-model'

# Mode granular configuration for deployment

config = dict(
    #assume_role_arn=exec_arn,
    execution_role_arn=exec_arn,
    bucket_name="mlflow-sagemaker-delme",
    image_url=image_url,
    region_name=region,
    archive=False,
    instance_type="ml.t2.medium",
    instance_count=1,
    synchronous=True,
    timeout_seconds=3600,
    variant_name="mlflow-variant-1",
    tags={"training_timestamp": "2023-10-23"},
)

client = get_deploy_client("sagemaker:/" + region)

client.create_deployment(
    #you can use any name you want to see in Sagemaker"
    name=deployment_name,
    model_uri=model_uri,
    flavor="python_function",
    config=config,
)

In [52]:
# Simple deployment
client.create_deployment(
    name=deployment_name,
    model_uri=model_uri,
    config={
        "image_url": image_url,
        "execution_role_arn": exec_arn
    }
)

2023/10/25 16:53:34 INFO mlflow.sagemaker: Using the python_function flavor for deployment!
2023/10/25 16:53:35 INFO mlflow.sagemaker: No model data bucket specified, using the default bucket
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
2023/10/25 16:53:35 INFO mlflow.sagemaker: Default bucket `mlflow-sagemaker-us-west-2-415275363822` already exists. Skipping creation.
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
2023/10/25 16:53:36 INFO mlflow.sagemaker: tag response: {'ResponseMetadata': {'RequestId': '6CH3BT0PBBD9W50K', 'HostId': 'BqKoghSAELgq7+6WkdtkdJ6NaJ9x0OT3+YaX+6qobfnbtHatothNKvIz2ulzizqop5dUEWTkZd4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'BqKoghSAELgq7+6WkdtkdJ6NaJ9x0OT3+YaX+6qobfnbtHatothNKvIz2ulzizqop5dUEWTkZd4=', 'x-amz-request-id': '6CH3BT0PBBD9W50K', 'date': 'Wed, 25 Oct 2023

{'name': 'mlflow-sagemaker-deploy-02', 'flavor': 'python_function'}

### Deploy using CLI (optional)

In [ ]:
!mlflow deployments create --target sagemaker:/us-west-2/arn:aws:415275363822:role/assumed_role \
        --name my-deployment \
        --model-uri /mlruns/0/abc/model \
        --flavor python_function\
        -C execution_role_arn=arn:aws:456:role/execution_role \
        -C bucket_name=my-s3-bucket \
        -C image_url=1234.dkr.ecr.us-east-1.amazonaws.com/mlflow-test:1.23.1 \
        -C region_name=us-east-1 \
        -C archive=False \
        -C instance_type=ml.m5.4xlarge \
        -C instance_count=1 \
        -C synchronous=True \
        -C timeout_seconds=300 \
        -C variant_name=prod-variant-1 \
        -C vpc_config='{"SecurityGroupIds": ["sg-123456abc"], \
        "Subnets": ["subnet-123456abc"]}' \
        -C data_capture_config='{"EnableCapture": True, \
        'InitalSamplingPercentage': 100, 'DestinationS3Uri": 's3://my-bucket/path', \
        'CaptureOptions': [{'CaptureMode': 'Output'}]}'
        -C env='{"DISABLE_NGINX": "true", "GUNICORN_CMD_ARGS": ""--timeout 60""}' \
        -C tags='{"training_timestamp": "2022-11-01T05:12:26"}' \

### Deploy your own container using SageMaker (Optional)
With Amazon SageMaker, you can package your own algorithms that can than be trained and deployed in the SageMaker environment. This notebook will guide you through an example that shows you how to build a Docker container for SageMaker and use it for training and inference.

By packaging an algorithm in a container, you can bring almost any code to the Amazon SageMaker environment, regardless of programming language, environment, framework, or dependencies.

![image](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/5daada592797be296c1aa7e7964e73900473dddd/advanced_functionality/scikit_bring_your_own/stack.png)

Note: SageMaker now includes a pre-built scikit container. We recommend the pre-built container be used for almost all cases requiring a scikit algorithm. However, this example remains relevant as an outline for bringing in other libraries to SageMaker as your own container.

Here is a more comprehensive <a>[code sample](https://github.com/aws/amazon-sagemaker-examples/blob/main/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)</a> for your reference.

In [47]:
# Create a SageMaker session
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()
# this is just a dummy location. The model is called with train data. We use the current notebook as dummy train data.
uri = sess.upload_data(train_data_file_path)

sm_model = sage.estimator.Estimator(
    image_url,
    exec_arn,
    1,
    "ml.c4.2xlarge",
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess,
)
# Run the train program because it is expected
sm_model.fit(uri)

# Deploy the model.
predictor = sm_model.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

INFO:sagemaker:Creating training-job with name: mlflow-pyfunc-2023-10-25-16-33-55-739


2023-10-25 16:33:56 Starting - Starting the training job...
2023-10-25 16:34:13 Starting - Preparing the instances for training.........
2023-10-25 16:35:57 Downloading - Downloading input data........Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.8/dist-packages/mlflow/models/container/__init__.py", line 55, in _init
    _train()
  File "/usr/local/lib/python3.8/dist-packages/mlflow/models/container/__init__.py", line 228, in _train
    raise Exception("Train is not implemented.")
Exception: Train is not implemented.

2023-10-25 16:37:25 Training - Training image download completed. Training in progress.
2023-10-25 16:37:25 Uploading - Uploading generated training model
2023-10-25 16:37:25 Failed - Training job failed


UnexpectedStatusException: Error for Training job mlflow-pyfunc-2023-10-25-16-33-55-739: Failed. Reason: AlgorithmError: , exit code: 1

## Verify SageMaker endpoint by sending a query

In [17]:
!aws sagemaker list-endpoints

{
    "Endpoints": [
        {
            "EndpointName": "mlflow-sagemaker-deploy-01",
            "EndpointArn": "arn:aws:sagemaker:us-west-2:415275363822:endpoint/mlflow-sagemaker-deploy-01",
            "CreationTime": "2023-10-23T23:31:02.728000+00:00",
            "LastModifiedTime": "2023-10-23T23:37:22.784000+00:00",
            "EndpointStatus": "InService"
        },
        {
            "EndpointName": "huggingface-pytorch-tgi-inference-2023-08-07-18-58-14-306",
            "EndpointArn": "arn:aws:sagemaker:us-west-2:415275363822:endpoint/huggingface-pytorch-tgi-inference-2023-08-07-18-58-14-306",
            "CreationTime": "2023-08-07T18:58:15.087000+00:00",
            "LastModifiedTime": "2023-08-07T19:06:56.207000+00:00",
            "EndpointStatus": "Failed"
        },
        {
            "EndpointName": "huggingface-pytorch-tgi-inference-2023-08-07-18-57-55-411",
            "EndpointArn": "arn:aws:sagemaker:us-west-2:415275363822:endpoint/huggingface-pytorch-tgi

In [53]:
import pandas as pd
import numpy as np
import json
import boto3

global app_name
global region
region = 'us-west-2'

def check_status(app_name):
    sage_client = boto3.client('sagemaker', region_name=region)
    endpoint_description = sage_client.describe_endpoint(EndpointName=deployment_name)
    endpoint_status = endpoint_description['EndpointStatus']
    return endpoint_status

def query_endpoint(app_name, input_json):
    client = boto3.session.Session().client('sagemaker-runtime', region)

    response = client.invoke_endpoint(
        EndpointName = app_name,
        Body = input_json,
        ContentType = 'application/json'#'; format=pandas-split',
        )

    preds = response['Body'].read().decode('ascii')
    preds = json.loads(preds)
    print('Received response: {}'.format(preds))
    return preds

# Check endpoint status
print('Application status is {}'.format(check_status(deployment_name)))

#Let's create a test array that we'll use to test our model
arr_predict = np.random.randn(2,3)

# Create test data and make inference from endpoint
query_input = pd.DataFrame(arr_predict).to_dict(orient='split')
print(query_input)

data = {"dataframe_split": query_input}

byte_data = json.dumps(data).encode('utf-8')

predictions = query_endpoint(app_name=deployment_name, input_json=byte_data)
print(predictions)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Application status is InService
{'index': [0, 1], 'columns': [0, 1, 2], 'data': [[1.276052127730331, -1.0328733404598451, -1.1620296910871757], [-1.596586482223551, 0.5859066495738939, -0.4625818296076901]]}
Received response: {'predictions': [1, 1]}
{'predictions': [1, 1]}


## Mangeing deployment

You can query an existing deployment by name and update the deployed model by replacing it with the output of a different run. Specify the run ID associated with a different training run.

In [54]:
deployment_info = client.get_deployment(name=deployment_name)
print(f"MLflow SageMaker Deployment status is: {deployment_info['EndpointStatus']}")

MLflow SageMaker Deployment status is: InService


In [ ]:
run_id2 = "<run-id2>"
new_model_uri = "runs:/" + run_id2 + "/model"

deployment_client.update_deployment(
  name=deployment_name,
  model_uri=new_model_uri,
  config={
    "image_url": image_ecr_url,
    "mode": "replace",
  }
)

In [55]:
## Clean up

In [23]:
client.delete_deployment(deployment_name)

2023/10/23 23:43:27 INFO mlflow.sagemaker: Deleted endpoint with arn: arn:aws:sagemaker:us-west-2:415275363822:endpoint/mlflow-sagemaker-deploy-01
2023/10/23 23:43:27 INFO mlflow.sagemaker: Waiting for the delete operation to complete...
2023/10/23 23:43:27 INFO mlflow.sagemaker: Deletion is still in progress. Current endpoint status: Deleting
2023/10/23 23:43:32 INFO mlflow.sagemaker: The deletion operation completed successfully with message: "The SageMaker endpoint was deleted successfully."
2023/10/23 23:43:32 INFO mlflow.sagemaker: Cleaning up unused resources...
2023/10/23 23:43:32 INFO mlflow.sagemaker: Deleted associated endpoint configuration with arn: arn:aws:sagemaker:us-west-2:415275363822:endpoint-config/mlflow-sagemaker-deploy-01-config-q5maywp6soc7dx6jp8otgq
2023/10/23 23:43:33 INFO mlflow.sagemaker: Deleted associated model with arn: arn:aws:sagemaker:us-west-2:415275363822:model/mlflow-sagemaker-deploy-01-model-nga-dp8qsrabekinvzanuuw
